# Sentiment Analysis

In [ ]:
%matplotlib inline
from bs4 import BeautifulSoup
# import urllib2
from urllib.request import urlopen
import urllib
import json
import csv
import time
import pickle
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
import sys
sns.set_style("whitegrid")
sns.set_context("poster")

In [ ]:
file = open('SentiWordNet_3.0.0_20130122.txt', 'r')

In [ ]:
lines = file.readlines()
print(lines)

In [ ]:
lines[len(lines)-2].split()

In [ ]:
word_sent = {}
for i in range(27,len(lines)-1):
    curr_list = lines[i].split()
    word=curr_list[4].rsplit('#', 1)[0]
    word_sent.setdefault(word, []).append((float(curr_list[2]), float(curr_list[3])))

In [ ]:
lines[4000]

In [ ]:
word_sent

In [ ]:
from numpy import mean

for key in word_sent:
#     print(key)
    if len(word_sent[key])>1:#print key
        word_sent[key]=tuple(map(mean, zip(*word_sent[key])))
    else:
        word_sent[key]=word_sent[key][0]

In [ ]:
word_sent

In [ ]:
tweets=pd.read_csv("english_tweets.csv")
tweets_translated=pd.read_csv("translatedTweets.csv")

In [ ]:
tweets.head()

In [ ]:
tweets_translated.head()

In [ ]:
print(len(tweets_translated))
print(len(tweets))

In [ ]:
import nltk
nltk.download('all')

from pattern import en
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA

punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [ ]:
#this converts tweet to nouns and adjectives in their basic forms
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns_or_adj=[]
    print(thetext.split())
    for sentence in parse(thetext, tokenize=True, lemmata=True).split():
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS','NN','NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns_or_adj.append(token[4].lower()) #make all lowercase
    return nouns_or_adj

In [ ]:
def sentiGraderNew(tweet):
    pos=[]
    neg=[]
    for word in get_parts(row.Text):
        if word in word_sent.keys():
            pos.append(word_sent[word][0])
            neg.append(word_sent[word][1])
    return pos,neg

In [ ]:
sampletweettext=tweets.iloc[3].Text
sampletweettext

In [ ]:
try:
    print(get_parts(sampletweettext)) 
except:
    StopIteration

# English Tweets

In [ ]:
%%time
mean_pos=[]
mean_neg=[]
total_pos=[]
total_neg=[]
for i in range(len(tweets)):
    row=tweets.iloc[i]
    pos=sentiGraderNew(row.Text)[0]
    neg=sentiGraderNew(row.Text)[1]
    mean_pos.append(np.mean(pos) if len(pos)>0 else 0)
    mean_neg.append(np.mean(neg) if len(neg)>0 else 0)
    total_pos.append(np.sum(x>0.5 for x in pos))
    total_neg.append(np.sum(x>0.5 for x in neg))

# Non-English Tweets

In [ ]:
%%time
#translated tweets
mean_pos_trans=[]
mean_neg_trans=[]
total_pos_trans=[]
total_neg_trans=[]
for i in range(len(tweets_translated)):
    row=tweets_translated.iloc[i]
    pos=sentiGraderNew(row.Text)[0]
    neg=sentiGraderNew(row.Text)[1]
    mean_pos_trans.append(np.mean(pos) if len(pos)>0 else 0)
    mean_neg_trans.append(np.mean(neg) if len(neg)>0 else 0)
    total_pos_trans.append(np.sum(x>0.5 for x in pos))
    total_neg_trans.append(np.sum(x>0.5 for x in neg))

In [ ]:
eng_tweets_with_sent=tweets
eng_tweets_with_sent['mean_pos']=mean_pos
eng_tweets_with_sent['mean_neg']=mean_neg
eng_tweets_with_sent['total_pos']=total_pos
eng_tweets_with_sent['total_neg']=total_neg
eng_tweets_with_sent.to_csv("eng_tweets_with_sent.csv", index=False)

In [ ]:
# trans_tweets_with_sent

In [ ]:
frames=[eng_tweets_with_sent]
all_tweets_with_sent=pd.concat(frames)

In [ ]:
sentiment_features=all_tweets_with_sent.groupby(['Company',"Series"]).mean()
sentiment_features.to_csv("sentiment_features", index=False)

In [ ]:
sentiment_features.head()